## Model making

In [1]:
import torch
import os
import transformers
from ast import literal_eval
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig
from datasets import Dataset
import json
import pandas as pd
import random
import numpy as np
import evaluate
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling
from sklearn.utils.class_weight import compute_class_weight
from torch.nn import CrossEntropyLoss

# 1. 설정: pandas 출력 옵션 및 시드 고정
pd.set_option('display.max_columns', None)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42)
# 2. 모델 및 토크나이저 로드 (8-bit 양자화)
model_name = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # QLoRA는 4bit 양자화를 사용
    # load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # 계산 precision (float16 또는 bfloat16 사용 가능)
    bnb_4bit_use_double_quant=True,       # 이중 양자화 활성화
    bnb_4bit_quant_type="nf4"             # NF4 양자화 타입
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # BitsAndBytesConfig 추가
    device_map="auto",
    trust_remote_code = True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 3. PEFT 설정 (LoRA)
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj'],
    bias="none",
    task_type="CAUSAL_LM",
)

# model = get_peft_model(model, lora_config)


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [00:12<00:00,  2.52s/it]


## Data Processing

In [2]:
def make_dataset(dataset, prompts, system_prompt):
    records = []
    for _, row in dataset.iterrows():
        problems = literal_eval(row['problems'])
        record = {
            'id': row['id'],
            'paragraph': row['paragraph'],
            'question': problems['question'],
            'choices': problems['choices'],
            'answer': problems.get('answer', None),
            "question_plus": problems.get('question_plus', ''),
            'klue' : row.get('klue', None),
            'question_type' : row.get('question_type', None)
        }
        records.append(record)
            
    # Convert to DataFrame
    df = pd.DataFrame(records)
    dataset = Dataset.from_pandas(df)
    processed_dataset = []

    for i in range(len(dataset)):
        choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(dataset[i]["choices"])])
        if dataset[i]['question_type'] == '이해형':
            prompt = prompts.이해형
        elif dataset[i]['question_type'] == '기타':
            prompt = prompts.기타
        elif dataset[i]['question_type'] == '사실형':
            prompt = prompts.사실형
        elif dataset[i]['question_type'] == '추론형':
            prompt = prompts.추론형
        else:
            prompt = prompts.나열형
        user_message = prompt.format(
            paragraph=dataset[i]["paragraph"],
            question=dataset[i]["question"],
            question_plus=dataset[i]["question_plus"],
            question_type=dataset[i]['question_type'],
            choices=choices_string,
        )
        # chat message 형식으로 변환
        processed_dataset.append(
            {
                "id": dataset[i]["id"],
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message},
                    {"role": "assistant", "content": f"{dataset[i]['answer']}"}
                ],
                "label": dataset[i]['answer'],
                'type' : dataset[i]['question_type']
            }
        )
    processed_dataset = Dataset.from_pandas(pd.DataFrame(processed_dataset))

    def formatting_prompts_func(example):
        output_texts = []
        for i in range(len(example["messages"])):
            output_texts.append(
                tokenizer.apply_chat_template(
                    example["messages"][i],
                    tokenize=False,
                )
            )
        return output_texts

    def tokenize(element):
        outputs = tokenizer(
            formatting_prompts_func(element),
            truncation=False,
            padding=False,
            return_overflowing_tokens=False,
            return_length=False,
        )
        return {
            "input_ids": outputs["input_ids"],
            "attention_mask": outputs["attention_mask"],
        }

    # 데이터 토큰화
    tokenized_dataset = processed_dataset.map(
        tokenize,
        remove_columns=list(processed_dataset.features),
        batched=True,
        num_proc=1,
        load_from_cache_file=True,
        desc="Tokenizing",
    )

    # 데이터 분리
    tokenized_dataset = tokenized_dataset.filter(lambda x: len(x["input_ids"]) <= 2048)  
    tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.10, seed=42)

    train_dataset = tokenized_dataset['train']
    eval_dataset = tokenized_dataset['test']


    train_dataset_token_lengths = [len(train_dataset[i]["input_ids"]) for i in range(len(train_dataset))]
    print(f"max token length: {max(train_dataset_token_lengths)}")
    print(f"min token length: {min(train_dataset_token_lengths)}")
    print(f"avg token length: {np.mean(train_dataset_token_lengths)}")

    # 데이터 확인
    return train_dataset, eval_dataset

In [3]:
from prompts import user_prompts
system_prompt = """지시에 따라 주어진 문제의 정답을 고르세요."""
prompts = user_prompts
dataset = pd.read_csv('datas/train+klue.csv')
train_dataset, eval_dataset = make_dataset(dataset,prompts, system_prompt)
# 여기서 프롬프트만 바꿔서 데이터셋을 만들기

Filter: 100%|██████████| 2031/2031 [00:01<00:00, 1631.54 examples/s]


max token length: 1561
min token length: 247
avg token length: 699.9135194307609


In [4]:
print(tokenizer.decode(train_dataset['input_ids'][5]))

<s><|im_start|> system
지시에 따라 주어진 문제의 정답을 고르세요.<|im_end|> 
<|im_start|> user

문제 유형:
기타

지문:
본문 1:
“전하, 게다가 우리들의 왕국에는 신께 도움이 되지 않는 불편함이 존재합니다. 바로 우리 백성들 중 많은 이들이 그대의 백성들이 가져오는 왕국의 상품과 물건을 간절히 원하고 있다는 것, 그런데 그대의 백성들은 자신들의 탐욕스러운 욕망을 만족시키기 위해 자유민이자 해방된 나의 백성들을 잡아가고 있다는 것입니다. 심지어 귀족과 왕의 친척까지도 잡아가 우리들의 왕국에 있는 백인들에게 팔고 있다는 것입니다.”
콩고의 아폰소 1세 국왕이 포르투갈의 주앙 3세 국왕에게 보낸 편지, 1526
출처 2:
“이번 원정에 많은 비용이 들었기에 빈 손으로 돌아간다면 합리적이지 못한 일이 될 것이다. 우리의 [주된] 바람은 신을 섬기는 것과 콩고 국왕을 기쁘게 하는 것이지만, 그럼에도 불구하고 콩고 국왕으로 하여금 노예가 됐건 구리가 됐건 상아가 됐건 배를 채워야 한다는 사실을 우리들의 이름으로 이해시켜야만 한다.”
포르투갈 마누엘 국왕의 콩고에 있는 사절에게 보낸 편지, 1512

질문:
편지에 설명된 상호 작용은 다음 중 어떤 맥락에서 가장 잘 이해되는가?

선택지:
1 - 포르투갈의 서아프리카 해안 탐험
2 - 사하라 이남 아프리카에서의 가톨릭 선교 활동
3 - 사하라 이남 아프리카의 국가 형성
4 - 사하라 이남 아프리카의 노예 무역 발전

1, 2, 3, 4, 5 중에 하나를 정답으로 고르세요.
문제를 풀이할 때, 반드시 지문을 참고하세요.
반드시 지문에서 정답의 근거를 찾으세요.

출력 형식:
근거 : 답변을 도출한 텍스트 # 정답 번호
반드시 출력 형식을 지키세요.
<|im_end|> 
<|im_start|> assistant
근거 : 내용 형식입니다. 본문에서는 콩고 왕국이 포르투갈에 의해 불법적인 인신매매와 탐욕으로 고통받고 있음을 강조하고, 이에 따른 불만이 표현되고 있습니다. 이를 통해 

## Train phase

In [5]:
import numpy as np
import evaluate
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
def preprocess_logits_for_metrics(logits, labels):
    logits = logits if not isinstance(logits, tuple) else logits[0]
    logit_idx = [tokenizer.vocab["1"],
                    tokenizer.vocab["2"],
                    tokenizer.vocab["3"],
                    tokenizer.vocab["4"], 
                    tokenizer.vocab["5"]]
    logits = logits[:,:, logit_idx]
    logits = logits[:,-1, :] # -2: answer token, -1: eos token
    return logits


    # metric 계산 함수
def compute_metrics(evaluation_result):
    logits, labels = evaluation_result
    int_output_map = {"1": 0, "2": 1, "3": 2, "4": 3, "5": 4}


    # 토큰화된 레이블 디코딩
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    labels = [label.split('#')[-1].strip() for label in labels]
    labels = [int_output_map.get(label, -1) for label in labels] 

    # 소프트맥스 함수를 사용하여 로그트 변환
    probs = torch.nn.functional.softmax(torch.tensor(logits, dtype=torch.float32), dim=-1)

    predictions = np.argmax(probs, axis=-1)

    # 정확도 계산
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")

    return {"accuracy": acc["accuracy"], "f1": f1["f1"]}


In [6]:
response_template = "assistant"
data_collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer,
)

sft_config = SFTConfig(
    do_train=True,
    do_eval=True,
    lr_scheduler_type="cosine",
    max_seq_length=2048,
    output_dir=f"./outputs + {model_name.split('/')[-1]}",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=1,
    max_steps=600,
    eval_steps=60,
    eval_strategy="steps",  # eval도 step 단위로 수행
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="steps",
    save_steps=60,
    save_total_limit=3,
    save_only_model=True,
    report_to="none",
    fp16=True,
    gradient_checkpointing=True,
    load_best_model_at_end=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics = preprocess_logits_for_metrics,
    args=sft_config,
    packing=False,
    peft_config = lora_config
)
torch.cuda.empty_cache()
sft_config.gradient_checkpointing = True
sft_config.use_cache = False

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


In [7]:
sft_config.gradient_checkpointing = True
sft_config.use_cache = False
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1
60,0.924800,0.823112,0.500000,0.141986
120,0.783700,0.788458,0.549020,0.216025
180,0.876400,0.763525,0.490196,0.212784
240,0.763200,0.757368,0.524510,0.216474
300,0.803300,0.750436,0.539216,0.252740
360,0.785800,0.746614,0.480392,0.198501
420,0.743300,0.744389,0.504902,0.221262
480,0.791000,0.743685,0.465686,0.211615
540,0.779300,0.743244,0.441176,0.204852
600,0.775200,0.743085,0.436275,0.203230


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=600, training_loss=0.8215067124366761, metrics={'train_runtime': 9332.6493, 'train_samples_per_second': 0.514, 'train_steps_per_second': 0.064, 'total_flos': 2.1482248155168768e+17, 'train_loss': 0.8215067124366761, 'epoch': 2.6272577996715927})

## Inference Phase

In [8]:
from ast import literal_eval
from datasets import Dataset

def make_test_dataset(test_df, prompt, system_prompt):
    # Flatten the JSON dataset
    records = []
    for _, row in test_df.iterrows():
        problems = literal_eval(row['problems'])
        record = {
            'id': row['id'],
            'paragraph': row['paragraph'],
            'question': problems['question'],
            'choices': problems['choices'],
            'answer': problems.get('answer', None),
            "question_plus": problems.get('question_plus', ''),
            'klue': row.get('klue', None),
            'question_type': row.get('question_type', None)
        }
        records.append(record)

    # Convert to DataFrame
    df = pd.DataFrame(records)
    dataset = Dataset.from_pandas(df)
    
    # test_dataset를 빈 리스트로 초기화
    test_dataset = []

    # dataset의 각 항목에 대해 처리
    for i in range(len(dataset)):
        # choices_string 생성
        choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(dataset[i]["choices"])])
        
        # question_type에 맞는 prompt 선택
        if dataset[i]['question_type'] == '이해형':
            prompt = prompts.이해형
        elif dataset[i]['question_type'] == '기타':
            prompt = prompts.기타
        elif dataset[i]['question_type'] == '사실형':
            prompt = prompts.사실형
        elif dataset[i]['question_type'] == '추론형':
            prompt = prompts.추론형
        else:
            prompt = prompts.나열형

        # user_message 생성
        user_message = prompt.format(
            paragraph=dataset[i]["paragraph"],
            question=dataset[i]["question"],
            question_plus=dataset[i]["question_plus"],
            question_type=dataset[i]['question_type'],
            choices=choices_string,
        )

        # len_choices는 choices의 길이로 계산
        len_choices = len(dataset[i]["choices"])

        # test_dataset에 항목 추가
        test_dataset.append(
            {
                "id": dataset[i]["id"],  # dataset[i]에서 'id' 가져오기
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message},
                ],
                "label": dataset[i].get('answer', None),  # 'answer'가 없으면 None
                'type': dataset[i].get('question_type', None),
                "len_choices": len_choices,
            }
        )

    return test_dataset


In [9]:
from torch.utils.data import DataLoader
from prompts import user_prompts

test_df = pd.read_csv('datas/test_processing_4.csv')
prompts = user_prompts
test_dataset = make_test_dataset(test_df, prompts, system_prompt)


In [10]:
test_dataset[130]

{'id': 'generation-for-nlp-130',
 'messages': [{'role': 'system', 'content': '지시에 따라 주어진 문제의 정답을 고르세요.'},
  {'role': 'user',
   'content': '\n문제 유형:\n사실형\n\n지문:\n한  평도  채  안  되는  구 멍가게 는 중풍으로 쓰러져 정상적 건강   상태가 아니었던 아버지의 유일한 수입원이자 생존 이유였다.   때문에 ㉠그 구멍가게에 대한 아버지의 몰두와 자존심은 각별했다 . 한번은 내가 아버지가 가게를 잠깐 비운 사이에 겉에 허연  인공 설탕 가루를 묻힌 ‘미키대장군’ 이라는 캐러멜을 하나 아무  생각 없이 널름 집어먹은 적이 있었다.  하나에 이 원 ,  다섯 개에   십 원이었다.  잠시 뒤에 돌아온 아버지는 단박에 그 사실을 알아 채고는 불같이 화를 내며 내 목덜미에 당수를 한 대 세게 내려 꽂는 것이었다.  그 캐러멜 갑 안에 미키대장군이 몇 개 들어  있는지조차 훤히 꿰차고 있는 아버지였다 . ―이런 민한 종간나래 !  얌생이처럼 기러케 쏠라닥질을 허자면   이 가게 안에 뭐이가 하나 제대로 남아나겠니 ,  응 ? 그러고 나서는 좀 머쓱했는지 입이 한 발쯤 튀어나와 뾰로통 해서 서 있는 내게 미키대장군 네 개를 집어 내미는 거였다 .   어차피 짝이 맞아야 파니까니,  하면서 억지로 내 손아귀에 쥐어 주었다.  ㉡나는 그 무허가 불량 식품인 캐러멜 네 개가 끈끈하게   녹아내릴 때까지 먹지 않고 쥔 채 서 있었다 . ―늴큼 털어 넣지 못하겠니,  으잉? 목덜미에 아버지의 가벼운 당수를 한 대 더 얹은 다음에야  한입에 털어 넣고 돌아서 나왔다.  아버지도 가게 일을 수월하게   보려면 잔심부름꾼인 나를 무시하고는 아쉬울 때가 많을 터였다 .   워낙 짧은 밑천으로 가게를 꾸려 가자니 아버지는 물건 구색을  맞추느라 하루에도 많을 때는 세 번까지 시장통 도매상으로 정부미   포대를 거머쥐고 종종걸음을 쳐야 했고 ,  막내인 나는 번번이 

In [11]:
tokenizer.padding_side = 'left'


# 배치 데이터 로더를 위한 collate_fn
def collate_fn(batch):
    ids = [item["id"] for item in batch]
    messages = [item["messages"] for item in batch]
    labels = [item.get("label", None) for item in batch]  # 라벨이 존재할 경우만 가져옴
    return ids, messages, labels

## 1. Test dataset inference


In [12]:
from accelerate import Accelerator
import torch
import gc
import pandas as pd
from tqdm import tqdm

# Accelerator 객체 생성
accelerator = Accelerator()

# GPU 메모리 비우기
torch.cuda.empty_cache()
gc.collect()

# 데이터 로더 설정
batch_size = 2  # 배치 크기 설정
dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

# 모델과 데이터를 accelerator로 준비
model = accelerator.prepare(model)
dataloader = accelerator.prepare(dataloader)

generated_infer_results = []
model.eval()
with torch.no_grad():
    for batch in tqdm(dataloader):
        ids, messages, labels = batch

        # 텍스트 생성을 위한 입력 데이터 준비
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            padding=True,
            return_attention_mask=True 
              # 배치 크기에 맞게 패딩 추가
        )

        # GPU로 이동 (inputs가 Tensor일 경우 바로 이동)
        inputs = inputs.to(model.device)  # Tensor로 직접 처리

        # 모델 생성
        outputs = model.generate(
            inputs,
            max_new_tokens=128,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,  # 종료 토큰 설정
        )

        # 결과 디코딩
        generated_texts = tokenizer.batch_decode(
            outputs[:, inputs.shape[1]:], skip_special_tokens=True
        )

        # 결과 저장
        for _id, generated_text, label in zip(ids, generated_texts, labels):
            generated_infer_results.append({
                "id": _id,
                "answer": generated_text,
                "label": label  # 실제 라벨이 있다면 포함
            })

# 결과를 DataFrame으로 저장
generated_infer_results = pd.DataFrame(generated_infer_results)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
  0%|          | 0/435 [00:00<?, ?it/s]

100%|██████████| 435/435 [10:29:56<00:00, 86.89s/it]   


In [13]:
generated_infer_results.to_csv('output.csv', index = False)

In [14]:
import re
def extract_last_digit(s):
    match = re.search(r'\d$', s)  # 문자열 끝에서 숫자 하나만 매칭
    return match.group() if match else None  # 매칭된 숫자를 반환, 없으면 None 반환

result = generated_infer_results
# 데이터프레임에 적용
result['text'] = result['answer']
result['answer'] = result['answer'].apply(lambda x: x.split('#')[-1])
result['answer'] = result['answer'].apply(extract_last_digit)
result['answer'] = result['answer'].fillna(1)
result['answer'] = result['answer'].apply(lambda x: int(x))
submission = result[['id', 'text', 'answer']]
submission['answer'].value_counts()

answer
1    405
2    167
3    155
4    117
5     24
6      1
Name: count, dtype: int64

In [31]:
submission[['id','answer']].to_csv('output.csv', index = False)
submission['answer'].value_counts()

answer
1    406
2    167
3    155
4    117
5     24
Name: count, dtype: int64

### 2. Eval dataset으로 inference
#### 프롬프트를 바꿔가며 몇개 맞추나 보기

In [13]:
tokenizer.padding_side = 'left'
batch_size = 8  # 배치 크기 설정
dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=collate_fn)

generated_infer_results = []

with torch.inference_mode():
    for batch in tqdm(dataloader):
        ids, messages, labels = batch

        # 텍스트 생성을 위한 입력 데이터 준비
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            padding=True,  # 배치 크기에 맞게 패딩 추가
        )

        # GPU로 이동 (inputs가 Tensor일 경우 바로 이동)
        inputs = inputs.to(model.device)  # Tensor로 직접 처리

        # 모델 생성
        outputs = model.generate(
            inputs,
            max_new_tokens=150,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,  # 종료 토큰 설정
        )

        # 결과 디코딩
        generated_texts = tokenizer.batch_decode(
            outputs[:, inputs.shape[1]:], skip_special_tokens=True
        )
        

        # 결과 저장
        for _id, generated_text, label in zip(ids, generated_texts, labels):
            generated_infer_results.append({
                "id": _id,
                "answer": generated_text,
                "label": label  # 실제 라벨이 있다면 포함
            })

# 결과를 DataFrame으로 저장
generated_infer_results = pd.DataFrame(generated_infer_results)

100%|██████████| 26/26 [16:46<00:00, 38.71s/it]


In [14]:
from copy import deepcopy

import re
def extract_last_digit(s):
    match = re.search(r'\d$', s)  # 문자열 끝에서 숫자 하나만 매칭
    return match.group() if match else None  # 매칭된 숫자를 반환, 없으면 None 반환

result = deepcopy(generated_infer_results)
# 데이터프레임에 적용
result['text'] = result['answer']
result['predict'] = result['answer'].apply(lambda x: x.split('#')[-1])
result['predict'] = result['predict'].apply(extract_last_digit)
result['predict'] = result['predict'].fillna(1)
result['predict'] = result['predict'].apply(lambda x: int(x))

result = result[['text', 'predict', 'label',]]
result['question_type'] = eval_dataset['type']
result['맞았는지 여부'] = result['predict'] == result['label']


In [24]:
result['text'][6]

'이 문서는 네덜란드의 특권과 헌장에 기초하여, 다양한 신앙을 가진 집단 간의 포용과 공동체의 선을 위한 법적 기준을 설정하는 것을 목표로 두고, 네덜란드 식민지의 다양한 구성원들을 수용하고자 하는 의지를 표현하고 있다.'

In [ ]:
문제 아이디  //   만든 문장   //  만든문장에서뽑은정답  //  정답  //  문제유형  //  맞았는지틀렸는지 


df[df['문제유형'] == '추론형']['맞았는지틀렸는지']